In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

# Lê o arquivo CSV do GitHub, base:
# Programa Bolsa Família (PBF) - valor total em R$ dos benefícios em dezembro
pbf_df = pd.read_csv("https://raw.githubusercontent.com/gugalbdias/ICDP/main/ipeadata_PBF.csv", header=1)

#Remover coluna vazia e transpor colunas dos anos
pbf_df = pbf_df.drop(columns=["Unnamed: 12"], errors='ignore')
pbf_df = pbf_df.melt(id_vars=["Sigla", "Código", "Estado"], value_vars=pbf_df.columns[3:],var_name="Year", value_name="pbf")



# Lê o arquivo CSV do GitHub, base:
# IDHM Ajustado à Desigualdade (IDHAD)
idhad_df = pd.read_csv("https://raw.githubusercontent.com/gugalbdias/ICDP/main/ipeadata_IDHAD.csv", header=1)

# Remover coluna vazia e transpor colunas dos anos
idhad_df = idhad_df.drop(columns=["Unnamed: 13"], errors='ignore')
idhad_df = idhad_df.melt(id_vars=["Sigla", "Código", "Estado"], value_vars=idhad_df.columns[3:],var_name="Year", value_name="idhad")



# Lê o arquivo CSV do GitHub, base:
# Esperança de vida ao nascer (EVN)
evn_df = pd.read_csv("https://raw.githubusercontent.com/gugalbdias/ICDP/main/ipeadata_EVN.csv", header=1)

# Remover coluna vazia e transpor colunas dos anos
evn_df = evn_df.drop(columns=["Unnamed: 13"], errors='ignore')
evn_df = evn_df.melt(id_vars=["Sigla", "Código", "Estado"], value_vars=evn_df.columns[3:],var_name="Year", value_name="evn")


# Lê o arquivo CSV do GitHub, base:
# Percentual de 6 a 14 anos de idade na escola
perc_e_df = pd.read_csv("https://raw.githubusercontent.com/gugalbdias/ICDP/main/ipeadata_PercCriancasEscola.csv", header=1)

# Remover coluna vazia e transpor colunas dos anos:
perc_e_df = perc_e_df.drop(columns=["Unnamed: 13"], errors='ignore')
perc_e_df = perc_e_df.melt(id_vars=["Sigla", "Código", "Estado"], value_vars=perc_e_df.columns[3:],var_name="Year", value_name="perc_escola")


,Sigla,Código,Estado,Year,perc_escola
0,AC,12,Acre,2012,97.14
1,AL,27,Alagoas,2012,97.49
2,AM,13,Amazonas,2012,96.75
3,AP,16,Amapá,2012,96.46
4,BA,29,Bahia,2012,97.98


In [2]:
# Unificando a base de dados:
df_unificado = pd.merge(perc_e_df, pbf_df, on=["Sigla", "Código", "Estado","Year"], how='outer')
df_unificado = pd.merge(df_unificado, evn_df, on=["Sigla", "Código", "Estado","Year"], how='outer')
df_unificado = pd.merge(df_unificado, idhad_df, on=["Sigla", "Código", "Estado","Year"], how='outer')

df_unificado.head()

,Sigla,Código,Estado,Year,perc_escola,pbf,evn,idhad
0,AC,12,Acre,2012,97.14,13541254.0,72.47,0.502
1,AL,27,Alagoas,2012,97.49,64992708.0,69.96,0.475
2,AM,13,Amazonas,2012,96.75,57464682.0,70.85,0.512
3,AP,16,Amapá,2012,96.46,9427588.0,72.77,0.542
4,BA,29,Bahia,2012,97.98,262704562.0,72.39,0.490


In [4]:
# Criando dicionário das regiões e incluindo coluna correspondente na base
regioes = {
    'Acre': 'Norte',
    'Alagoas': 'Nordeste',
    'Amapá': 'Norte',
    'Amazonas': 'Norte',
    'Bahia': 'Nordeste',
    'Ceará': 'Nordeste',
    'Distrito Federal': 'Centro-Oeste',
    'Espírito Santo': 'Sudeste',
    'Goiás': 'Centro-Oeste',
    'Maranhão': 'Nordeste',
    'Mato Grosso': 'Centro-Oeste',
    'Mato Grosso do Sul': 'Centro-Oeste',
    'Minas Gerais': 'Sudeste',
    'Pará': 'Norte',
    'Paraíba': 'Nordeste',
    'Paraná': 'Sul',
    'Pernambuco': 'Nordeste',
    'Piauí': 'Nordeste',
    'Rio de Janeiro': 'Sudeste',
    'Rio Grande do Norte': 'Nordeste',
    'Rio Grande do Sul': 'Sul',
    'Rondônia': 'Norte',
    'Roraima': 'Norte',
    'Santa Catarina': 'Sul',
    'São Paulo': 'Sudeste',
    'Sergipe': 'Nordeste',
    'Tocantins': 'Norte'
}

df_unificado["Região"] = df_unificado["Estado"].map(regioes)


df_unificado


,Sigla,Código,Estado,Year,perc_escola,pbf,evn,idhad,Região
0,AC,12,Acre,2012,97.14,13541254.0,72.47,0.502,Norte
1,AL,27,Alagoas,2012,97.49,64992708.0,69.96,0.475,Nordeste
2,AM,13,Amazonas,2012,96.75,57464682.0,70.85,0.512,Norte
3,AP,16,Amapá,2012,96.46,9427588.0,72.77,0.542,Norte
4,BA,29,Bahia,2012,97.98,262704562.0,72.39,0.490,Nordeste
...,...,...,...,...,...,...,...,...,...
265,RS,43,Rio Grande do Sul,2021,99.58,80017146.0,72.84,0.642,Sul
266,SC,42,Santa Catarina,2021,99.16,29029828.0,74.60,0.670,Sul
267,SE,28,Sergipe,2021,98.73,57103069.0,70.85,0.522,Nordeste
268,SP,35,São Paulo,2021,99.42,339241964.0,73.61,0.654,Sudeste


In [5]:
# Adicionar colunas com os logaritmos das variáveis
df_unificado['log_pe'] = np.log(df_unificado['perc_escola'])
df_unificado['log_evn'] = np.log(df_unificado['evn'])
df_unificado['log_idhad'] = np.log(df_unificado['idhad'])
df_unificado['log_pbf'] = np.log(df_unificado['pbf'])

df_unificado.head()


,Sigla,Código,Estado,Year,perc_escola,pbf,evn,idhad,Região,log_pe,log_evn,log_idhad,log_pbf
0,AC,12,Acre,2012,97.14,13541254.0,72.47,0.502,Norte,4.576153,4.283173,-0.689155,16.421251
1,AL,27,Alagoas,2012,97.49,64992708.0,69.96,0.475,Nordeste,4.579750,4.247924,-0.744440,17.989786
2,AM,13,Amazonas,2012,96.75,57464682.0,70.85,0.512,Norte,4.572130,4.260565,-0.669431,17.866681
3,AP,16,Amapá,2012,96.46,9427588.0,72.77,0.542,Norte,4.569128,4.287304,-0.612489,16.059151
4,BA,29,Bahia,2012,97.98,262704562.0,72.39,0.490,Nordeste,4.584763,4.282068,-0.713350,19.386541


In [7]:
# Remover linhas com valores ausentes
df_cleaned = df_unificado.dropna(subset=['log_pe', 'log_idhad', 'log_pbf', 'log_evn'])

# Substituir valores infinitos por NaN
df_cleaned.replace([np.inf, -np.inf], np.nan, inplace=True)

# Remover linhas com NaN
df_cleaned.dropna(subset=['log_pe', 'log_idhad', 'log_pbf', 'log_evn'], inplace=True)

# Realizar a regressão com os dados limpos
model = sm.OLS(df_cleaned['log_evn'], 
               sm.add_constant(df_cleaned[['log_pe', 'log_idhad', 'log_pbf']]))
results = model.fit()

print(results.summary())



                            OLS Regression Results                            
Dep. Variable:                log_evn   R-squared:                       0.606
Model:                            OLS   Adj. R-squared:                  0.601
Method:                 Least Squares   F-statistic:                     122.5
Date:                Mon, 28 Aug 2023   Prob (F-statistic):           4.62e-48
Time:                        20:03:06   Log-Likelihood:                 578.16
No. Observations:                 243   AIC:                            -1148.
Df Residuals:                     239   BIC:                            -1134.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          4.4929      0.979      4.588      0.0

C:\Users\gusta\AppData\Local\Temp\ipykernel_14484\1146442265.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned.replace([np.inf, -np.inf], np.nan, inplace=True)
C:\Users\gusta\AppData\Local\Temp\ipykernel_14484\1146442265.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned.dropna(subset=['log_pe', 'log_idhad', 'log_pbf', 'log_evn'], inplace=True)
